In [1]:
import copy
import json

In [1]:
#Outside of Python, from command line:
#BLAST+ software (https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/)

#makeblastdb -in ~/databases/DFAM/Dfam_embl.fasta -dbtype 'nucl'

#blastn -query ./NEAT1_gene_all.fasta -db ~/databases/DFAM/Dfam_embl.fasta -task blastn /
#-num_threads 30 -out ./NEAT1_on_Dfam_nodust.txt -max_target_seqs 100000 /
#-outfmt "6 qseqid sseqid length pident qstart qend sstart send evalue bitscore" -max_hsps 10 /
#-soft_masking false -dust no


In [5]:
#Make a dictionary with sequences of NEAT1 orthologs
seq = {}
seq_file = './NEAT1_gene_all.fasta'
with open(seq_file) as file1:
    fl = 0
    for line in file1:
        if line.startswith('>') and fl == 0:
            l = []
            name = line.rstrip().lstrip('>')
            fl = 1
        elif line.startswith('>') and fl == 1:
            seq[name] = ''.join(l)
            l = []
            name = line.rstrip().lstrip('>')
        elif not line.startswith('>'):
            l.append(line.strip())
    seq[name] = ''.join(l)

In [2]:
# Dictionary of genomes' IDs in which NEAT1 was identified  
neat1 = {}
with open('Metadata') as file1:
    for line in file1:
        dat = line.strip().split('\t')
        if not line.startswith('#') and dat[3] == '1':
            dat = line.strip().split('\t')
            neat1[dat[0]] = {}


Parsing results of blast search against of Dfam database with the 80-80-80 rule (80 percent coverage of a TE, over at least 80bp and 80% of nucleotide identity)

In [7]:
out = open('./NEAT1_dfam_blast_matches_80-80-80','w')
out_seq = open('./NEAT1_all_matched_TE.fasta','w')
d = {}
with open('./NEAT1_on_Dfam.txt') as file1:
    for line in file1:
        dat = line.strip().split('\t')
        if int(dat[2]) > 80 and float(dat[3]) > 80:
            loclength = int(dat[1].split('_')[-1])
            if (int(dat[2]) / loclength) > 0.8:
                out.write(line)
                if dat[0] not in d:
                    d[dat[0]] = {}
                d[dat[0]][dat[1]] = [loclength,int(dat[4]),int(dat[5])]
                out_seq.write('>' + dat[0] + '|' + dat[1] + '|' + dat[4] + '-' + dat[5] + '\n')
                out_seq.write(seq[dat[0]][int(dat[4]):int(dat[5])].upper() + '\n')
                
out.close()
out_seq.close()

In [12]:
# Make a subset of TEs annotations which are non-overlapped (20bp maximum as an overlap between 
#two annotations is allowed) 
ran_dd = {}
out = open('./TE/NEAT1_non_overlaped_TE.fasta','w')
out_stat = open('./TE/Percentage_of_NEAT1_in_repeats','w')
out_stat.write('#Organism\tCoverage_TE\tCoverage_self_alignment\n')
for genome in neat1:
    if genome in d:
        x = sorted(d[genome].keys(), key=lambda kv: d[genome][kv][0],reverse = True)
        offset = 20
        ran = [0 for i in range(length[genome])]
        ran_d = {}
        for TE in x:
            if ran:
                start = d[genome][TE][1]
                stop = d[genome][TE][2]
                fl = 0
                s = sum(ran[start:stop])
                if s < offset:
                    for i in range(start,stop):
                        ran[i] = 1
                    ran_d[TE] = d[genome][TE]

            else:
                for i in range(d[genome][TE][1],d[genome][TE][2]):
                    ran[i] = 1
                ran_d[TE] = d[genome][TE]
        for item in ran_d:
            out.write('>' + genome + '|' + item + '|' + str(ran_d[item][1]) + '-' + str(ran_d[item][2]) + '\n')
            out.write(seq[genome][ran_d[item][1]:ran_d[item][2]].upper() + '\n')
        ran_dd[genome] = copy.deepcopy(ran_d)

        out_stat.write('\t'.join([genome, str(round((sum(ran)/length[genome])*100,2)),str(round((sum(self_align[genome])/length[genome])*100,2))]) + '\n')
out.close()
out_stat.close()

In [13]:
json.dump(ran_dd, open("./TE/NEAT1_non_overlapping_TE.json","w"))
json.dump(d, open("./TE/NEAT1_all_hits_TE.json","w"))